<a href="https://colab.research.google.com/github/eth0-02/Astro-Theme-Creek/blob/master/SP_FINAL_SCRIPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# === Colab: Source CSVs -> LONG outputs with EXACT columns (no extras), MonthName, unit-labelled metrics ===
!pip -q install pandas numpy

import pandas as pd, numpy as np, re, os
from calendar import month_abbr
from pathlib import Path
from google.colab import files

OUT_DIR = Path("outputs_bi_clean"); OUT_DIR.mkdir(parents=True, exist_ok=True)

# ----------------------- helpers -----------------------
def _find_timestep_cols(df, base):
    """Find columns like base_1..base_N (case-insensitive), ordered by index."""
    pat = re.compile(fr"^{base}_(\d+)$", flags=re.IGNORECASE)
    found = []
    for c in df.columns:
        m = pat.match(c)
        if m: found.append((int(m.group(1)), c))
    found.sort(key=lambda x: x[0])
    return [c for _, c in found]

def _month_means(row, cols):
    """Return 12 monthly means across all years (averaging Jan, Feb, ...)."""
    if not cols:
        return pd.Series([np.nan]*12, index=range(1,13))
    vals = pd.to_numeric(row[cols], errors="coerce").to_numpy(dtype="float64")
    months = (np.arange(vals.size) % 12) + 1
    return pd.DataFrame({"m": months, "v": vals}).groupby("m")["v"].mean().reindex(range(1,13))

def _non_timestep_cols(df):
    """Keep everything except H_#, overflow_#, R_#, balance_#, and MonthNumber/MonthofYear (from source)."""
    step_pat = re.compile(r"^(H|overflow|R|balance)_\d+$", flags=re.IGNORECASE)
    drop_src_months = {"monthnumber","monthofyear"}
    return [c for c in df.columns if not step_pat.match(c) and c.lower() not in drop_src_months]

def _choose_level(df, fname):
    """Detect level by columns then filename: Sub-Basin > Basin > County > Generic."""
    cols = {c.lower() for c in df.columns}
    fn = fname.lower()
    if "sub_hybas_id" in cols: return "Sub-Basin"
    if ("sub" in fn and "basin" in fn) or "sub-basin" in fn or "sub_basin" in fn: return "Sub-Basin"
    if "basin_name" in cols: return "Basin"
    if "basin_hybas_id" in cols and "sub_hybas_id" not in cols: return "Basin"
    if "basin" in fn: return "Basin"
    if "county" in cols or "county_name" in cols or "county" in fn: return "County"
    return "Generic"

def _unify_fid_keep(df):
    """If both FID and fid exist: keep the one that appears first, fill from the other, drop the duplicate."""
    if "FID" in df.columns and "fid" in df.columns:
        first_is_FID = list(df.columns).index("FID") < list(df.columns).index("fid")
        tgt, src = ("FID","fid") if first_is_FID else ("fid","FID")
        df[tgt] = df[tgt].where(pd.notna(df[tgt]), df[src])
        return df.drop(columns=[src])
    return df

def _id_cols_present(df):
    """Columns that must never be coerced/filled."""
    want = {"hybas_id","sub_hybas_id","basin_hybas_id","fid","adm0_code"}
    return [c for c in df.columns if c.lower() in want]

def _metric_fill_spaces_to_zero(df, id_cols):
    """
    Coerce numeric-ish columns & fill NaN/'' -> 0 for metrics only.
    Leaves IDs, text/name, geometry, MonthName, Level untouched.
    """
    never = set(id_cols) | {"MonthName","Level","geometry"}
    def is_text(c): return ("name" in c.lower()) or ("geom" in c.lower())
    for c in df.columns:
        if c in never or is_text(c):
            continue
        s2 = pd.to_numeric(
            df[c].astype(str).str.strip().replace({"": np.nan, "None": np.nan, "nan": np.nan}, regex=False),
            errors="coerce"
        )
        if s2.notna().sum()==0 and df[c].dtype==object:
            continue
        df[c] = s2.fillna(0)
    return df

def _drop_dup_names(df):
    """Remove duplicate column names (keep first occurrence)."""
    return df.loc[:, ~pd.Index(df.columns).duplicated(keep="first")]

def _unique_outpath(base_dir: Path, base_name: str) -> Path:
    """Create unique path if file already exists: Name.csv, Name (2).csv, ..."""
    p = base_dir / base_name
    if not p.exists(): return p
    i = 2
    stem, ext = os.path.splitext(base_name)
    while True:
        q = base_dir / f"{stem} ({i}){ext}"
        if not q.exists(): return q
        i += 1

# ----------------------- core transform -----------------------
def transform_source_to_long(upload_name: str) -> Path:
    src = Path(upload_name)
    base = pd.read_csv(src)
    base = _unify_fid_keep(base)

    # timestep detection and years
    Hc = _find_timestep_cols(base, "H")
    Oc = _find_timestep_cols(base, "overflow")
    Rc = _find_timestep_cols(base, "R")
    Bc = _find_timestep_cols(base, "balance")
    steps = max(len(Hc), len(Oc), len(Rc), len(Bc))
    years = int(steps/12) if steps else 0

    level = _choose_level(base, src.name)
    passthrough = _non_timestep_cols(base)

    # build LONG
    rows = []
    for _, r in base.iterrows():
        Hm = _month_means(r, Hc)  # metres
        Om = _month_means(r, Oc)  # metres
        Rm = _month_means(r, Rc)  # metres
        Bm = _month_means(r, Bc)  # metres

        # SA for volumes; if missing -> NaN -> filled to 0 later (metrics only)
        try: SA_val = float(r.get("SA"))
        except: SA_val = np.nan

        Om3 = Om * SA_val
        Rm3 = Rm * SA_val
        Bm3 = Bm * SA_val

        for m in range(1, 13):
            rec = {
                "Level": level,
                "YearsAveraged": years,
                "MonthName": month_abbr[m],
                # unit-labelled monthly means (ONLY additions you requested)
                "H (m)": Hm.loc[m],
                "R (M3)": Rm3.loc[m],
                "balance (M3)": Bm3.loc[m],
                "overflow (M3)": Om3.loc[m],
            }
            # pass through all original non-timestep columns EXACTLY as-is
            for c in passthrough:
                rec[c] = r.get(c, np.nan)
            rows.append(rec)

    df = pd.DataFrame.from_records(rows)
    df = _drop_dup_names(df)
    df = _unify_fid_keep(df)

    # enforce BASIN_HYBAS_ID rule
    if level in {"County","Basin"}:
        drop = [c for c in df.columns if c.lower()=="basin_hybas_id"]
        if drop: df = df.drop(columns=drop)

    # fill metrics only (IDs/text untouched)
    ids = _id_cols_present(df)
    df = _metric_fill_spaces_to_zero(df, ids)

    # choose output name
    if level == "County":
        out_path = _unique_outpath(OUT_DIR, "County Output BI.csv")
    elif level == "Basin":
        out_path = _unique_outpath(OUT_DIR, "Basin Output BI.csv")
    elif level == "Sub-Basin":
        out_path = _unique_outpath(OUT_DIR, "Sub-Basin Output BI.csv")
    else:
        out_path = _unique_outpath(OUT_DIR, f"{src.stem}__Generic Output BI.csv")

    df.to_csv(out_path, index=False)
    files.download(str(out_path))
    print(f"[{src.name}] → {out_path.name} | Level: {level} | YearsAveraged: {years}")
    return out_path

# ----------------------- optional: quick validation -----------------------
def validate_one_id_per_level(source_path: Path, output_path: Path, level_hint: str):
    """Pick one ID row, recompute monthly means from source, compare with output, and save a diff CSV."""
    src = pd.read_csv(source_path)
    out = pd.read_csv(output_path)

    # choose an ID column to match on
    for id_col in ["SUB_HYBAS_ID","HYBAS_ID","BASIN_HYBAS_ID"]:
        if id_col in src.columns and id_col in out.columns:
            break
    else:
        print(f"[{level_hint}] No common ID column to validate.")
        return

    # pick first row with non-null SA (to avoid zero volumes), fallback to first row
    if "SA" in src.columns and src["SA"].notna().any():
        sample_row = src[src["SA"].notna()].iloc[0]
    else:
        sample_row = src.iloc[0]
    sample_id = sample_row[id_col]

    # recompute from source
    Hc = _find_timestep_cols(src, "H")
    Oc = _find_timestep_cols(src, "overflow")
    Rc = _find_timestep_cols(src, "R")
    Bc = _find_timestep_cols(src, "balance")
    Hm = _month_means(sample_row, Hc)
    Om = _month_means(sample_row, Oc)
    Rm = _month_means(sample_row, Rc)
    Bm = _month_means(sample_row, Bc)
    SA_val = float(sample_row.get("SA")) if "SA" in src.columns else np.nan

    calc = pd.DataFrame({
        "MonthName": [month_abbr[m] for m in range(1,13)],
        "H (m) [calc]": Hm.values,
        "R (M3) [calc]": (Rm*SA_val).values,
        "balance (M3) [calc]": (Bm*SA_val).values,
        "overflow (M3) [calc]": (Om*SA_val).values,
    })

    out_sel = out[out[id_col]==sample_id][["MonthName","H (m)","R (M3)","balance (M3)","overflow (M3)"]].copy()
    merged = pd.merge(calc, out_sel, on="MonthName", how="inner")
    for col in ["H (m)","R (M3)","balance (M3)","overflow (M3)"]:
        merged[f"{col} diff"] = merged[f"{col} [calc]"] - merged[col]

    val_path = OUT_DIR / f"VALIDATION — {level_hint} — {id_col}={sample_id}.csv"
    merged.to_csv(val_path, index=False)
    files.download(str(val_path))
    print(f"[{level_hint}] validation saved:", val_path.name)

# ----------------------- run -----------------------
print("Upload one or more SOURCE CSVs (e.g., Kenbasin_hydrosheds.csv, Kencounty_hydrosheds.csv, sub_basins_with_basin_attributes.csv)")
uploaded = files.upload()

# transform each uploaded file
produced = {}
for name in uploaded.keys():
    outp = transform_source_to_long(name)
    produced[outp.name] = outp

# optional: auto-validate one sample per level if we can find a matching source file among uploads
# (match by level name in output file)
for out_name, out_path in produced.items():
    if "County Output BI" in out_name:
        # try to find a county-like source among uploads
        cand = next((k for k in uploaded.keys() if "county" in k.lower()), None)
        if cand: validate_one_id_per_level(Path(cand), out_path, "County")
    elif "Basin Output BI" in out_name:
        cand = next((k for k in uploaded.keys() if "basin" in k.lower()), None)
        if cand: validate_one_id_per_level(Path(cand), out_path, "Basin")
    elif "Sub-Basin Output BI" in out_name:
        cand = next((k for k in uploaded.keys() if "sub" in k.lower()), None)
        if cand: validate_one_id_per_level(Path(cand), out_path, "Sub-Basin")

print("Done.")
